In [36]:
from keras.models import Model
from keras.layers import Dropout, Dense, Input, LSTM, concatenate, ConvLSTM2D, Conv2D, Lambda, Reshape
from keras.losses import Huber
from keras.optimizers import Adam
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.nn import softmax, leaky_relu
from tensorflow import expand_dims, einsum
import tensorflow as tf

In [55]:
class Sgcn_Lstm():
    def __init__(self, train_x, train_y, valid_x, valid_y, AD, AD2, lr=0.0001, epoach=200, batch_size=10):
        self.train_x = train_x
        self.train_y = train_y
        self.valid_x = valid_x
        self.valid_y = valid_y
        self.AD = AD
        self.AD2 = AD2
        self.lr = lr
        self.epoach =epoach
        self.batch_size = batch_size
        self.num_joints = 25

    def sgcn(self, Input):
        x = tf.keras.layers.Conv2D(filters=64, kernel_size=(1,1), strides=1, activation='relu')(Input)
        x = Dropout(0.25)(x)
        gcn_1 = tf.keras.layers.Lambda(lambda x: tf.einsum('vw,ntwc->ntvc', x[0], x[1]))([self.AD, x])
        y = tf.keras.layers.Conv2D(filters=64, kernel_size=(1,1), strides=1, activation='relu')(Input)
        y = Dropout(0.25)(y)
        gcn_2 = tf.keras.layers.Lambda(lambda x: tf.einsum('vw,ntwc->ntvc', x[0], x[1]))([self.AD2, y])
        gcn = concatenate([gcn_1, gcn_2], axis=-1)                                                                                                                                   
        
        x = tf.keras.layers.Conv2D(filters=128, kernel_size=(1,1), strides=1, activation='relu')(gcn)
        x = Dropout(0.25)(x)
        gcn_1 = tf.keras.layers.Lambda(lambda x: tf.einsum('vw,ntwc->ntvc', x[0], x[1]))([self.AD, x])
        y = tf.keras.layers.Conv2D(filters=128, kernel_size=(1,1), strides=1, activation='relu')(gcn)
        y = Dropout(0.25)(y)
        gcn_2 = tf.keras.layers.Lambda(lambda x: tf.einsum('vw,ntwc->ntvc', x[0], x[1]))([self.AD2, y])
        gcn = concatenate([gcn_1, gcn_2], axis=-1)
        
        gcn = tf.keras.layers.Reshape(target_shape=(-1,gcn.shape[2]*gcn.shape[3]))(gcn)
        return gcn

    def Lstm(self,x):
        rec = LSTM(80, return_sequences=True)(x)
        rec = Dropout(0.25)(rec)
        rec1 = LSTM(40, return_sequences=True)(rec)
        rec1 = Dropout(0.25)(rec1)
        rec2 = LSTM(40, return_sequences=True)(rec1)
        rec2 = Dropout(0.25)(rec2)
        rec3 = LSTM(80)(rec2)
        rec3 = Dropout(0.25)(rec3)
        output = Dense(1, activation = 'linear')(rec3)
        return output

    def build(self):
        seq_input = Input(shape=(None, self.train_x.shape[2], self.train_x.shape[3]), batch_size=None)
        sgcn_layer = self.sgcn(seq_input)
        lstm_sgcn_layer = self.Lstm(sgcn_layer)
        self.model = Model(seq_input, lstm_sgcn_layer)
        return self.model     

    def train(self):
        self.model.compile(loss=tf.keras.losses.Huber(delta=0.1), optimizer= Adam(learning_rate=self.lr))
        checkpoint = ModelCheckpoint("models/model_ex5.keras", monitor='val_loss', save_best_only=True, mode='auto', save_freq='epoch')
        
        history = self.model.fit(self.train_x, self.train_y, validation_data = (self.valid_x,self.valid_y), epochs=self.epoach, batch_size=self.batch_size, callbacks=[checkpoint])
        return history
    
    def prediction(self, data):
        y_pred = self.model.predict(data)
        return y_pred

# Train Begin

In [52]:
import numpy as np

from sklearn.model_selection import train_test_split
from keras.optimizers import *
from sklearn.model_selection import train_test_split

from math import sqrt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

from data_processing import Data_Loader
from graph import Graph
# from GCN.sgcn_lstm import Sgcn_Lstm

from sklearn.metrics import mean_squared_error, mean_absolute_error

# import argparse


In [53]:

random_seed = 42  # for reproducibility

# # Create the parser
# my_parser = argparse.ArgumentParser(description='List of argument')

# # Add the arguments
# my_parser.add_argument('--ex', type=str, default='Kimore_ex5', help='the name of exercise.', required=True)
# my_parser.add_argument('--lr', type=int, default= 0.0001, help='initial learning rate for optimizer.')
# my_parser.add_argument('--epoch', type=int, default= 1000, help='number of epochs to train.')
# my_parser.add_argument('--batch-size', type=int, default= 10, help='training batch size.')

# # Execute the parse_args() method
# args = my_parser.parse_args()

exercise = 'data/KIMORE/Kimore_ex5'
learning_rate = 0.0001
epoch = 1000
batch_size = 10

In [54]:
"""import the whole dataset"""
data_loader = Data_Loader(exercise)  # folder name -> Train.csv, Test.csv

"""import the graph data structure"""
graph = Graph(len(data_loader.body_part))

"""Split the data into training and validation sets while preserving the distribution"""
train_x, valid_x, train_y, valid_y = train_test_split(data_loader.scaled_x, data_loader.scaled_y, test_size=0.2, random_state = random_seed)

print("Training instances: ", len(train_x))
print("Validation instances: ", len(valid_x))

"""Train the algorithm"""
# algorithm = Sgcn_Lstm(train_x, train_y, graph.AD, graph.AD2, graph.bias_mat_1, graph.bias_mat_2, lr = args.lr, epoach=args.epoch, batch_size=args.batch_size)
algorithm = Sgcn_Lstm(train_x, train_y, valid_x, valid_y, graph.AD, graph.AD2, lr = learning_rate, epoach=epoch, batch_size=batch_size)
model = algorithm.build()
history = algorithm.train()

Training instances:  298
Validation instances:  75


ValueError: The filepath provided must end in `.keras` (Keras model format). Received: filepath=models/best_model_ex1.hdf5

In [ ]:
"""Test the model"""
y_pred = algorithm.prediction(valid_x)
y_pred = data_loader.sc2.inverse_transform(y_pred)
valid_y = data_loader.sc2.inverse_transform(valid_y) 

In [ ]:
"""Performance matric"""
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
test_dev = abs(valid_y-y_pred)
mean_abs_dev = np.mean(test_dev)
mae = mean_absolute_error(valid_y, y_pred)
rms_dev = sqrt(mean_squared_error(y_pred, valid_y))
mse = mean_squared_error(valid_y,y_pred) 
mape = mean_absolute_percentage_error(valid_y, y_pred)

print('Mean absolute deviation:', mae)
print('RMS deviation:', rms_dev)
print('MSE:', mse)
print('MAPE: ', mape)